In [10]:
# Importe el conjunto de datos de diabetes y divídalo en entrenamiento y prueba usando scikit-learn
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import K2Score
from pgmpy.estimators import BicScore

import pandas as pd

import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score

from despliegue.infer import inferencia

# db = load_diabetes()
# X = db.data
# y = db.target
# X_train, X_test, y_train, y_test = train_test_split(X, y)

df = pd.read_csv("train.csv", sep = ";")
test = pd.read_csv("test.csv", sep = ";")
#df = df.drop(df.columns[0], axis = 1 )

In [14]:
#Importe MLFlow para registrar los experimentos, el regresor de bosques aleatorios y la métrica de error cuadrático medio


# defina el servidor para llevar el registro de modelos y artefactos
mlflow.set_tracking_uri('http://localhost:5000')
# registre el experimento
experiment = mlflow.set_experiment("sklearnK2")

# Aquí se ejecuta MLflow sin especificar un nombre o id del experimento. MLflow los crea un experimento para este cuaderno por defecto y guarda las características del experimento y las métricas definidas. 
# Para ver el resultado de las corridas haga click en Experimentos en el menú izquierdo. 
with mlflow.start_run(experiment_id=experiment.experiment_id):
    # # defina los parámetros del modelo
    max_indegree = 1
    max_iter = int(1e3)
    # n_estimators = 400 
    # max_depth = 12
    # max_features = 8
    # # Cree el modelo con los parámetros definidos y entrénelo
    # rf = RandomForestRegressor(n_estimators = n_estimators, max_depth = max_depth, max_features = max_features)
    # rf.fit(X_train, y_train)
    # # Realice predicciones de prueba
    # predictions = rf.predict(X_test)
    scoring_method = K2Score(data=df)
    esth = HillClimbSearch(data=df)
    estimated_modelh = esth.estimate(
        scoring_method=scoring_method, max_indegree=max_indegree, max_iter=max_iter
    )

    from pgmpy.models import BayesianNetwork
    modelo = BayesianNetwork(list(estimated_modelh.edges()))

    df_fit = df[list(estimated_modelh.nodes())]

    from pgmpy.estimators import MaximumLikelihoodEstimator

    emv = MaximumLikelihoodEstimator(model=modelo, data=df_fit)
    modelo.fit(data=df_fit, estimator = MaximumLikelihoodEstimator) 

    from pgmpy.inference import VariableElimination

    #Modelo de inferencia
    infer = VariableElimination(modelo)

    test_fit = test[list(estimated_modelh.nodes())]

    predictions = inferencia(modelo, test_fit)
  
    # Registre los parámetros
    mlflow.log_param("max Indegree", max_indegree)
    mlflow.log_param("max Iter", max_iter)

    # mlflow.log_param("num_trees", n_estimators)
    # mlflow.log_param("maxdepth", max_depth)
    # mlflow.log_param("max_feat", max_features)
  
    # Registre el modelo
    mlflow.sklearn.log_model(modelo, "K2-model")
  
    # Cree y registre la métrica de interés
    y_test = test.loc[:,"target"]
    accuracy = accuracy_score(y_test, predictions, normalize=False)
    matriz_confusion = confusion_matrix(test.loc[:,"target"], predictions, labels=["Low", "Medium", "High", "Very High"]).ravel()
    sensibilidad = recall_score(test.loc[:, "target"], predictions, labels=["Low", "Medium", "High", "Very High"], average=None)
    tasa = accuracy_score(test.loc[:,"target"], predictions)

    mlflow.log_metric("Exactitud", tasa)
    mlflow.log_metric("accuracy", accuracy)
   
    print(accuracy)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
#Importe MLFlow para registrar los experimentos, el regresor de bosques aleatorios y la métrica de error cuadrático medio
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score

# defina el servidor para llevar el registro de modelos y artefactos
mlflow.set_tracking_uri('http://localhost:5000')
# registre el experimento
experiment = mlflow.set_experiment("sklearn-BIC")

# Aquí se ejecuta MLflow sin especificar un nombre o id del experimento. MLflow los crea un experimento para este cuaderno por defecto y guarda las características del experimento y las métricas definidas. 
# Para ver el resultado de las corridas haga click en Experimentos en el menú izquierdo. 
with mlflow.start_run(experiment_id=experiment.experiment_id):
    # # defina los parámetros del modelo
    max_indegree = 1
    max_iter = int(1e3)
    # n_estimators = 400 
    # max_depth = 12
    # max_features = 8
    # # Cree el modelo con los parámetros definidos y entrénelo
    # rf = RandomForestRegressor(n_estimators = n_estimators, max_depth = max_depth, max_features = max_features)
    # rf.fit(X_train, y_train)
    # # Realice predicciones de prueba
    # predictions = rf.predict(X_test)
    scoring_method = BicScore(data=df)
    esth = HillClimbSearch(data=df)
    estimated_modelh = esth.estimate(
        scoring_method=scoring_method, max_indegree=4, max_iter=int(1e4)
    )


    from pgmpy.models import BayesianNetwork
    modelo = BayesianNetwork(list(estimated_modelh.edges()))

    df_fit = df[list(estimated_modelh.nodes())]

    from pgmpy.estimators import MaximumLikelihoodEstimator

    emv = MaximumLikelihoodEstimator(model=modelo, data=df_fit)
    modelo.fit(data=df_fit, estimator = MaximumLikelihoodEstimator) 

    from pgmpy.inference import VariableElimination

    #Modelo de inferencia
    infer = VariableElimination(modelo)

    test_fit = test[list(estimated_modelh.nodes())]

    predictions = inferencia(modelo, test_fit)
  
    # Registre los parámetros
    mlflow.log_param("max Indegree", max_indegree)
    mlflow.log_param("max Iter", max_iter)

    # mlflow.log_param("num_trees", n_estimators)
    # mlflow.log_param("maxdepth", max_depth)
    # mlflow.log_param("max_feat", max_features)
  
    # Registre el modelo
    mlflow.sklearn.log_model(modelo, "BIC-model")
  
    # Cree y registre la métrica de interés
    y_test = test.loc[:,"target"]
    accuracy = accuracy_score(y_test, predictions, normalize=False)
    matriz_confusion = confusion_matrix(test.loc[:,"target"], predictions, labels=["Low", "Medium", "High", "Very High"]).ravel()
    sensibilidad = recall_score(test.loc[:, "target"], predictions, labels=["Low", "Medium", "High", "Very High"], average=None)
    tasa = accuracy_score(test.loc[:,"target"], predictions)

    mlflow.log_metric("Exactitud", tasa)
    mlflow.log_metric("accuracy", accuracy)
   


    #mlflow.log_metric("matriz de confusión", matriz_confusion)
    #mlflow.log_metric("sensibilidad", sensibilidad)
    mlflow.log_metric("accuracy", accuracy)
    print(accuracy)

  0%|          | 0/10000 [00:00<?, ?it/s]

MlflowException: Failed to convert metric value to float: can only convert an array of size 1 to a Python scalar

In [ ]:
#Importe MLFlow para registrar los experimentos, el regresor de bosques aleatorios y la métrica de error cuadrático medio
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score


from pgmpy.estimators import PC

df_pc = df.drop(df.columns[0], axis = 1 )

# defina el servidor para llevar el registro de modelos y artefactos
mlflow.set_tracking_uri('http://localhost:5000')
# registre el experimento
experiment = mlflow.set_experiment("sklearn-PC")

# Aquí se ejecuta MLflow sin especificar un nombre o id del experimento. MLflow los crea un experimento para este cuaderno por defecto y guarda las características del experimento y las métricas definidas. 
# Para ver el resultado de las corridas haga click en Experimentos en el menú izquierdo. 
with mlflow.start_run(experiment_id=experiment.experiment_id):
    # # defina los parámetros del modelo
    max_indegree = 4
    max_iter = int(1e4)



    est = PC(data=df_pc)
    estimated_model = est.estimate(variant="stable", max_cond_vars=4)

    grafo = [("cole_bilingue","desemp_ingles"),("cole_calendario","cole_bilingue"),("cole_calendario","desemp_ingles"),("cole_calendario","cole_jornada"),("desemp_ingles","target"),("desemp_ingles","fami_estratovivienda"),("fami_estratovivienda","fami_tieneinternet"),("fami_tieneautomovil","fami_estratovivienda"),("cole_jornada","fami_estratovivienda"),("cole_jornada","fami_tieneinternet"),("cole_jornada","estu_edad_cat"),("cole_jornada","target"),("fami_tieneinternet","fami_tienecomputador")]


    from pgmpy.models import BayesianNetwork
    modelo = BayesianNetwork(grafo)

    df_fit = df_pc[list(estimated_model.nodes())]

    from pgmpy.estimators import MaximumLikelihoodEstimator

    emv = MaximumLikelihoodEstimator(model=modelo, data=df_fit)
    modelo.fit(data=df_fit, estimator = MaximumLikelihoodEstimator) 

    from pgmpy.inference import VariableElimination

    #Modelo de inferencia
    infer = VariableElimination(modelo)

    test_fit = test[list(estimated_model.nodes())]

    predictions = inferencia(modelo, test_fit)
  
    # Registre los parámetros
    mlflow.log_param("max Indegree", max_indegree)
    mlflow.log_param("max Iter", max_iter)

    # mlflow.log_param("num_trees", n_estimators)
    # mlflow.log_param("maxdepth", max_depth)
    # mlflow.log_param("max_feat", max_features)
  
    # Registre el modelo
    mlflow.sklearn.log_model(modelo, "PC-model")
  
    # Cree y registre la métrica de interés
    y_test = test.loc[:,"target"]
    accuracy = accuracy_score(y_test, predictions, normalize=False)
    #matriz_confusion = confusion_matrix(test.loc[:,"target"], predictions, labels=["Low", "Medium", "High", "Very High"]).ravel()
    #sensibilidad = recall_score(test.loc[:, "target"], predictions, labels=["Low", "Medium", "High", "Very High"], average=None)
    tasa = accuracy_score(test.loc[:,"target"], predictions)

    mlflow.log_metric("Exactitud", tasa)
    mlflow.log_metric("accuracy", accuracy)
    #mlflow.log_metric("matriz de confusión", matriz_confusion)
    #mlflow.log_metric("sensibilidad", sensibilidad)
    print(accuracy)

  0%|          | 0/4 [00:00<?, ?it/s]

INFO:pgmpy:Reached maximum number of allowed conditional variables. Exiting


5326


In [12]:
#Importe MLFlow para registrar los experimentos, el regresor de bosques aleatorios y la métrica de error cuadrático medio
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score


from pgmpy.estimators import PC

# defina el servidor para llevar el registro de modelos y artefactos
mlflow.set_tracking_uri('http://localhost:5000')
# registre el experimento
experiment = mlflow.set_experiment("sklearn-Hibrido-1")

# Aquí se ejecuta MLflow sin especificar un nombre o id del experimento. MLflow los crea un experimento para este cuaderno por defecto y guarda las características del experimento y las métricas definidas. 
# Para ver el resultado de las corridas haga click en Experimentos en el menú izquierdo. 
with mlflow.start_run(experiment_id=experiment.experiment_id):
    # # defina los parámetros del modelo
    max_indegree = 4
    max_iter = int(1e4)

    grafo1 = [('fami_tieneautomovil', 'fami_tieneinternet'), ('fami_tieneinternet', 'fami_tienecomputador'), ('fami_tieneinternet', 'fami_estratovivienda'), ('estu_edad_cat', 'cole_jornada'), ('cole_jornada', 'target'), ('cole_naturaleza', 'target'), ('cole_naturaleza', 'cole_calendario'), ('cole_naturaleza', 'desemp_ingles'), ('cole_naturaleza', 'cole_jornada'), ('desemp_ingles', 'fami_tieneinternet'), ('desemp_ingles', 'target'), ('desemp_ingles', 'cole_calendario'), ('desemp_ingles', 'fami_tienecomputador'), ('desemp_ingles', 'cole_jornada'), ('fami_tienecomputador', 'target'), ('fami_tienecomputador', 'fami_estratovivienda'), ('cole_bilingue', 'cole_calendario'), ('cole_bilingue', 'desemp_ingles')]


    from pgmpy.models import BayesianNetwork
    mod_fit_mv= BayesianNetwork(grafo1)
    
    df_fit = df[list( mod_fit_mv.nodes())]

    from pgmpy.estimators import MaximumLikelihoodEstimator

    emv = MaximumLikelihoodEstimator(model=mod_fit_mv, data=df_fit)
    mod_fit_mv.fit(data=df_fit, estimator = MaximumLikelihoodEstimator) 

    from pgmpy.inference import VariableElimination

    #Modelo de inferencia
    infer = VariableElimination(modelo)
    test_fit = test[list(mod_fit_mv.nodes())]

    predictions = inferencia(mod_fit_mv, test_fit)
  
    # Registre los parámetros
    mlflow.log_param("max Indegree", max_indegree)
    mlflow.log_param("max Iter", max_iter)

    # mlflow.log_param("num_trees", n_estimators)
    # mlflow.log_param("maxdepth", max_depth)
    # mlflow.log_param("max_feat", max_features)
  
    # Registre el modelo
    mlflow.sklearn.log_model(modelo, "Hibrido1-model")
  
    # Cree y registre la métrica de interés
    y_test = test.loc[:,"target"]
    accuracy = accuracy_score(y_test, predictions, normalize=False)
    #matriz_confusion = confusion_matrix(test.loc[:,"target"], predictions, labels=["Low", "Medium", "High", "Very High"]).ravel()
    #sensibilidad = recall_score(test.loc[:, "target"], predictions, labels=["Low", "Medium", "High", "Very High"], average=None)
    tasa = accuracy_score(test.loc[:,"target"], predictions)

    mlflow.log_metric("Exactitud", tasa)
    mlflow.log_metric("accuracy", accuracy)
    #mlflow.log_metric("matriz de confusión", matriz_confusion)
    #mlflow.log_metric("sensibilidad", sensibilidad)
    print(accuracy)

5322


In [13]:
#Importe MLFlow para registrar los experimentos, el regresor de bosques aleatorios y la métrica de error cuadrático medio
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score


from pgmpy.estimators import PC

# defina el servidor para llevar el registro de modelos y artefactos
mlflow.set_tracking_uri('http://localhost:5000')
# registre el experimento
experiment = mlflow.set_experiment("sklearn-Hibrido-1")

# Aquí se ejecuta MLflow sin especificar un nombre o id del experimento. MLflow los crea un experimento para este cuaderno por defecto y guarda las características del experimento y las métricas definidas. 
# Para ver el resultado de las corridas haga click en Experimentos en el menú izquierdo. 
with mlflow.start_run(experiment_id=experiment.experiment_id):
    # # defina los parámetros del modelo
    max_indegree = 4
    max_iter = int(1e4)

    grafo2 = [('cole_bilingue','desemp_ingles'),('cole_calendario','cole_bilingue'),('cole_calendario','desemp_ingles'),('cole_calendario','cole_jornada'),('desemp_ingles','fami_estratovivienda'),('desemp_ingles','target'),('desemp_ingles','cole_jornada'),('cole_jornada','target'),('cole_jornada','fami_tieneautomovil'),('fami_tieneautomovil','fami_estratovivienda'),('cole_jornada','estu_edad_cat')]


    from pgmpy.models import BayesianNetwork
    mod_fit_mv= BayesianNetwork(grafo2)
    
    df_fit = df[list( mod_fit_mv.nodes())]

    from pgmpy.estimators import MaximumLikelihoodEstimator

    emv = MaximumLikelihoodEstimator(model=mod_fit_mv, data=df_fit)
    mod_fit_mv.fit(data=df_fit, estimator = MaximumLikelihoodEstimator) 

    from pgmpy.inference import VariableElimination

    #Modelo de inferencia
    infer = VariableElimination(modelo)
    test_fit = test[list(mod_fit_mv.nodes())]

    predictions = inferencia(mod_fit_mv, test_fit)
  
    # Registre los parámetros
    mlflow.log_param("max Indegree", max_indegree)
    mlflow.log_param("max Iter", max_iter)

    # mlflow.log_param("num_trees", n_estimators)
    # mlflow.log_param("maxdepth", max_depth)
    # mlflow.log_param("max_feat", max_features)
  
    # Registre el modelo
    mlflow.sklearn.log_model(modelo, "Hibrido1-model")
  
    # Cree y registre la métrica de interés
    y_test = test.loc[:,"target"]
    accuracy = accuracy_score(y_test, predictions, normalize=False)
    #matriz_confusion = confusion_matrix(test.loc[:,"target"], predictions, labels=["Low", "Medium", "High", "Very High"]).ravel()
    #sensibilidad = recall_score(test.loc[:, "target"], predictions, labels=["Low", "Medium", "High", "Very High"], average=None)
    tasa = accuracy_score(test.loc[:,"target"], predictions)

    mlflow.log_metric("Exactitud", tasa)
    mlflow.log_metric("accuracy", accuracy)
    #mlflow.log_metric("matriz de confusión", matriz_confusion)
    #mlflow.log_metric("sensibilidad", sensibilidad)
    print(accuracy)

5326
